## Introduction

### Traveling Salesman Problem
The traveling salesman problem is a classic optimization problem that seeks to find the most efficient route that connects a given set of points. I recently discovered a set of services built by the open-source mapping company, Mapzen, that make this complex problem easy to solve. Given a set of coordinates, the Mapzen [optimize route](https://mapzen.com/documentation/mobility/optimized/api-reference/) service uses road network data to produce a time-distance matrix betweent these points, and then calculates the route that minimizes the total travel time. This can be done for one of three modes of transportaion - pedestrian, bicycle, and car. They have a great [example](https://mapzen.com/blog/optimized-route/) with a cool map where they determine the optimal route to visit burrito 'dispensaries' in San Franscico.

I use this root optimization tool in conjunction Mapzen's [Search](https://mapzen.com/documentation/search/) Service, which uses open source data to geocode addresses, landmarks, and businesses. Using these two services together is really handy, because it allows the user to specify locations without knowing their lat/long coordinates.

### Extension to the TSP
The Mapzen optimize route service takes a set of points and finds the optimal route that a person should take to visit all of these points. However, what if we have multiple "salesmen"? How should the stops be split up between people and in what order should each person visit their stops?

The idea for this was spurred by a project I'm involved with at work, in which we are sending out multiple research assistants to conduct surveys at a dozen or so different sites in Oakland. In this case, it doesn't matter if one person conducts more surveys than other or who goes to which site - the goal is just to minimize the total time to get them all done. 

Perhaps a more interesting and relevant application (for me!), is the optimization of Sunday morning errands between my girlfriend, Celeste, and I. Say we're both starting and ending at our apartment in the Inner Richmond, SF and have 6 different places that we need to stop at. How should Celeste and I split up these errands so that we're done as quickly as possible? 

In the post below, I write a set of functions in Python that wrap these two Mapzen services to help me answer these questions. Additionally, I will use the Python package ```folium``` to create leaflet.js slippy maps to display the results of this optimzation problem. Folium has a number of built-in tilesets from OpenStreetMap, MapQuest, MapBox, and makes it really easy to build web maps in Python. Let's get started!

In [1]:
import requests
import pandas as pd
import itertools
import shapely
from shapely.geometry import Point
import geopandas as gpd
import json
import numpy as np
import folium
from numpy.random import RandomState, uniform
import time
import sys

%matplotlib inline
search_key='search-WWoh7Na'
matrix_key='matrix-gjM8nT3'

## Geocoding Locations with Mapzen's Search Tool
I first write a function that wraps Mapzen's Search tool in order to geocode (obtain the lat / long coordinates for) an address, landmark, or business. The function returns a dictionary containg the raw result output from Mapzen (includes stuff such as data source, geocoding confidence, neighborhood data, etc), as well as the crucial piece of information for me - a formatted set of xy coordinates. By default I return only 1 result (the top result), but if I were to use this tool for a different context, I would perhaps be interested in returning a set of results. Below I use this function to geocode two famous San Francisco landmarks - the Transamerica Pyramid and Sutro Tower.

In [2]:
def geocode_address_venue(text, key=search_key,params=None): #allow for additional Mapzen search parameters
    search_parameters={'api_key':key,'text':text,'size':1,'layers':'address,venue'}
    if params:
        search_parameters.update(params)
    url='http://search.mapzen.com/v1/search'
    r = requests.get(url, params=search_parameters)
    data=r.json()
    return {'raw':data,'coords':tuple(data['features'][0]['geometry']['coordinates'])}

In [3]:
transamerica=geocode_address_venue('Transamerica Pyramid, San Francisco, CA')['coords']
sutro=geocode_address_venue('Sutro Tower, San Francisco, CA')['coords']
print transamerica
print sutro

(-122.40303, 37.79465)
(-122.45285, 37.755246)


## Displaying Points on Leaflet Maps
Now let's put these points on a map. I write a plotting function below that takes a list of coordinates and displays them on a Leaflet map. By default I use the Stamen Watercolor tiles because I think they look really cool, but if something like OpenStreetMap is more useful, that can also be specfied. The full set of available tile layers can be found [here](https://github.com/python-visualization/folium). The function also takes an optional set of colors and point labels, a zoom-level, and a center location. My function calculates default values if not specified. 

The function is written in a way that it accepts either a list of point coordinates, or a list of lists of point coordinates. If the latter is specified, each sublist is treated as a group of coordinates and will be symbolized in the same color. The function was ultimately written this way to make it easy to distinguish locations that each "salesman" will visit in the optimization problem coming later. Additionally, I allow for the specification of polyline coordinates, which will be used to plot the optimize route between the points.

Below I map my two previously geocoded San Francisco landmarks on Stamen Toner tiles. Note that on the 'live' map if you click on the points you will see that they are labeled appropriately.

In [4]:
def plot_stops(coords,zoom_level=15,tiles='Stamen Watercolor',colors=None, labels=None,center_location=None,line_coords=None):
    #if not a list of lists, make it one
    coords=[[x] for x in coords] if all([type(x) is not list for x in coords]) else coords
    if labels: labels=[[x] for x in labels] if all([type(x) is not list for x in labels]) else labels
    
    #if line_coords:
        #polylines=[(x,y) for (y,x) in line_coords]
    
    #get all points as the flattened list
    all_points=[item for sublist in coords for item in sublist]
    #calculate start location as the mean x and mean y of all input points
    if center_location:
        center_location=[center_location[1],center_location[0]]
    else:
        center_location=np.array(all_points).mean(0).tolist()[::-1]
    
    #create a leaflet map, specifying center location, zoom level, and tiles
    map_1 = folium.Map(location=center_location,zoom_start=zoom_level,tiles=tiles)
    #specify a set of default colors
    colors=colors if colors else ['black','blue','red','green','purple','orange','pink','white'][:len(coords)]
    
    #loop through each point or grouping of points
    for c,coords in enumerate(coords):
        color=colors[c] #get color from color list

        if labels: sublabel_list=labels[c] #get labels from label list
        for i,stop_coord in enumerate(coords): 
            label=sublabel_list[i] if labels else None
            #Add point to map at specified coordinate 
            folium.Marker([stop_coord[1],stop_coord[0]], popup=label,
                           icon = folium.Icon(color = color)).add_to(map_1)

    if line_coords:
        asdf=[]
        for i,pline in enumerate(line_coords):
            polyline=[(x,y) for (y,x) in pline]
            color=colors[1:][i]
            folium.PolyLine(polyline,color=color).add_to(map_1)
            asdf.append((polyline,color))

    return map_1

In [5]:
sf_points=plot_stops([transamerica, sutro], labels=['Transamerica Pyramid','Sutro Tower'], 
            zoom_level=13, tiles='Stamen Toner',colors=['red','blue'])
sf_points

I also demonstrate how a list of list of points can be passed to my plotting function, resulting in each set of points getting symbolized in a different color. I write a function that generates random points within a polygon, and use that to generate 6 lists of 5 points located in San Francisco (boundaries read in from a geosjon). I plot them below. Here the point are labeled by there group number (1-6) and their point number within their group (1-5).

In [6]:
def gen_random_points(poly, n, random_seed=None):
    xmin,ymin,xmax,ymax=poly.bounds
    Points=[]
    i=0
    while len(Points)<=n:
        if random_seed:
            x,y=RandomState(random_seed+i).uniform(xmin,xmax), RandomState(random_seed+i+1).uniform(ymin,ymax)
        else:
            x,y=uniform(xmin,xmax),uniform(ymin,ymax)
        if Point(x,y).within(poly):
            Points.append((x,y))
        i+=1
    return Points

In [7]:
SF=gpd.read_file('SF.geojson').iloc[0]['geometry']
rand_points=[gen_random_points(SF,n=5) for t in range(6)]
labels=[['G'+str(t)+ '-'+'P'+str(i) for i,x in enumerate(l,1)] for t,l in enumerate(rand_points,1)]

sf_rand_points=plot_stops(rand_points,zoom_level=12,tiles='Stamen Toner',labels=labels)
sf_rand_points

## Determining Stop Order using Mapzen's Optimize Route Service 
Now that I have tools to geocode and visualize points, I'm ready write a function that uses the optimize route tool to determine the optimal order in which one person should visit a set of points. Later on, I will build off of this function to optimize stops for multiple people, but this is an initial building-block.

My function is written in a way that the start and end location are assumed to be the same, although this could easily be tweaked. The function takes a "home" location, a set of stops, and calculates the quickest route to go from home, to each of these stops, and back. It returns a dictionary of the raw output, as well as the pieces of information that are most relevant for my purposes - trip length, trip distance, optimized stop order, stop coordinates, and route coordinates.

Note that Mapzen uses the [Google Maps encoded polyline format](https://developers.google.com/maps/documentation/utilities/polylinealgorithm) to store a series of latitude, longitude coordinates as a single string (this is done to reduce the size of the route). However, for my purposes, I need the string decoded into a list of lat/long coordinates. Mapzen provides [code]((https://developers.google.com/maps/documentation/utilities/polylinealgorithm) that can be used to decode the string, which I use below.

My function then wraps the geocoding function I wrote above, so that the user can specify any combination of addresses, venues, and coordinates (if coordinates are specified they are not passed to the geocoding function).  It defaults to pedestrian mode of transportation, but can also optimize driving and biking routes.

I also allow the user to specify a set of stop labels, which will be returned as the ordered list of stops. Otherwise, the function defaults to the raw input that was passed to the function.

In [8]:
#six degrees of precision in valhalla
inv = 1.0 / 1e6;
#decode an encoded string
def decode(encoded):
    decoded = []
    previous = [0,0]
    i = 0
    #for each byte
    while i < len(encoded):
        #for each coord (lat, lon)
        ll = [0,0]
        for j in [0, 1]:
            shift = 0
            byte = 0x20
            #keep decoding bytes until you have this coord
            while byte >= 0x20:
                byte = ord(encoded[i]) - 63
                i += 1
                ll[j] |= (byte & 0x1f) << shift
                shift += 5
            #get the final value adding the previous offset and remember it for the next
            ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
            previous[j] = ll[j]
            #scale by the precision and chop off long coords also flip the positions so
            #its the far more standard lon,lat instead of lat,lon
        decoded.append([float('%.6f' % (ll[1] * inv)), float('%.6f' % (ll[0] * inv))])
    #hand back the list of coordinates
    return decoded

In [32]:
def optimize_stops(home,stops,costing='pedestrian', api_key=matrix_key,home_label='Home',
                  stop_labels=None):
    
    #geocode home and stops if not a coordinate
    home=home if type(home)==tuple else geocode_address_venue(home)
    stops=[geocode_address_venue(stop)['coords'] if type(stop)<>tuple else stop for stop in stops]
    
    #full set of points are list of points that start and end with the home location
    points=[home]+stops+[home]
    
    #define point labels
    names=[home_label]+(stop_labels if stop_labels else stops)+[home_label]
    
    #set up parameters to pass to mapzen function
    js={'locations':[{'lon':point[0], 'lat':point[1]} for point in points],
                   'costing':costing}
    params={'json':json.dumps(js),'api_key':api_key}
    url='https://matrix.mapzen.com/optimized_route'
    r = requests.get(url,params=params)
    raw=r.json()
    locs=raw['trip']['locations']
    
    #get the coordinates of the stops in their new optimized order
    new_point_order=[(locs[loc]['lon'],locs[loc]['lat']) for loc in range(len(locs))]
    
    #the new points returned from the mapzen tool need to be matched up to my original set of points passed
    #to the function, so that the point name order can be determined. The mapzen points are not exactly the same 
    #values as the original points passed (they are probably located to the nearest street or something). 
    #Therefore, I link the new points and the original points by rounding the coordinates to account for 
    #differences in input and output. This is then used to calculate the order of the point names
    new_point_order_rounded=[(round(x,3), round(y,3)) for (x,y) in new_point_order]
    original_points_rounded=[(round(x,3), round(y,3)) for (x,y) in points]
    point_order=[original_points_rounded.index(x) for x in new_point_order_rounded]
    name_order=[names[i] for i in point_order]
    
    raw_path=[raw['trip']['legs'][x]['shape'] for x in range(len(raw['trip']['legs']))]
    decode_paths=[tuple(item) for sublist in [decode(z) for z in raw_path] for item in sublist]
    
    #cleaned_path=[[(y,x) for (x,y) in decode(testline)] for z in raw_path]
    #together=[item for sublist in cleaned_path for item in sublist]
    #return a dictionary that contains raw mapzen output, trip time, trip distance, new ordered points, and 
    #new ordered point names
    return {'raw':raw, 
    'time':raw['trip']['summary']['time']/60,
    'length':raw['trip']['summary']['length'], 
    'points':new_point_order,
    'order':name_order,
    'line':decode_paths}

Below, I set up the parameters that I will pass to the optimize route function. I specify a list of 6 stops in my neighborhood that Celeste and I will need to visit as part of our Sunday morning errands (note that I use a combination of business names, addresses, and coordinates). I also specify the set of names that correspond to these stops, that will be used just to make the labeling easier. As much as I wish it were the case, our Sunday errands generally do not involve ice cream, pizza, bagels, and art museums. Nonetheless, for this example I've included my favorite desinations of these types as stops that we need to make.

Lastly, I specify our approximate home coordinates that we will be routing to and from.

In [33]:
stops=['Toy Boat Dessert Cafe, San Francisco, CA',
       'Pizzetta 211, San Francisco, CA', 
       'Arguello Super Market, San Francisco, CA',
       '4700 Geary Blvd, San Francisco, CA',
       (-122.465613,37.770016),
       '3519 California St, San Francisco, CA 94118']

stop_labels=['Toy Boat',
             'Pizzetta',
             'Arguello Market',
             'Lamps Plus',
             'de Young Museum',
             "Noah's Bagels"]

home=(-122.464186,37.779111)

I then apply this function to my set of stops and return the optimized stop order, the route time, and the route distance. I first optimize the route as a pedestrian and then as a bicyclist. As you can see, the stop order is slightly different for these two modes of transit, likely keeping the bike route on roads / paths that are better for biking.

In [11]:
walk_opt=optimize_stops(home, stops, stop_labels=stop_labels)
print walk_opt['order']
print str(walk_opt['time'])+' minutes'
print str(walk_opt['length'])+ ' km'

['Home', 'de Young Museum', 'Arguello Market', "Noah's Bagels", 'Toy Boat', 'Pizzetta', 'Lamps Plus', 'Home']
113 minutes
9.357 km


In [12]:
bike_opt=optimize_stops(home, stops, stop_labels=stop_labels,costing='bicycle')
print bike_opt['order']
print str(bike_opt['time'])+' minutes'
print str(bike_opt['length'])+ ' km'

['Home', 'Pizzetta', 'Lamps Plus', 'Toy Boat', "Noah's Bagels", 'Arguello Market', 'de Young Museum', 'Home']
29 minutes
11.08 km


## Determining Stops and Order with Multiple People

Now that I have a working function that wraps Mapzen's optimize route service, I am ready to extend it work with multiple people. The general approach is to first find the unique combinations that a list of stops can be split among a given number of people, and then determine in which of these combinations minimizes the maximum time of any 1 person. 

### Unique ways that stops can be split
I use a function adapted from [here](http://stackoverflow.com/a/39199937/3776938) to find the unique ways in that a list of N elements can be partitioned into K groups. This function is written so that the order of groups or of elements within a group does not matter. By this I mean that ```[['A','B'],['C','D']]``` is considered identical to ```[['C','D'],['A','B']]``` as well as to ```[['B','A'],['C','D']]```.

In [13]:
def sorted_k_partitions(seq, k):
    n = len(seq)
    working_partition = []

    def generate_partitions(i):
        if i >= n:
            yield list(map(tuple, working_partition))
        else:
            if n - i > k - len(working_partition):
                for part in working_partition:
                    part.append(seq[i])
                    for bar in generate_partitions(i + 1):
                        yield bar
                    part.pop()

            if len(working_partition) < k:
                working_partition.append([seq[i]])
                for bar in generate_partitions(i + 1):
                    yield bar
                working_partition.pop()

    result = generate_partitions(0)

    # Sort the parts in each partition in shortlex order and then by the length of each part, 
    #and then lexicographically
    result = [sorted(ps, key = lambda p: (len(p), p)) for ps in result]
    result = sorted(result, key = lambda ps: (map(len, ps), ps))
    return result

I demonstrate an application of the function below, showing the 7 unique ways that 4 stops can be split among 3 people. Obviously if I were to split these 4 stops among 4 people or among 1 person, there is only 1 possible solution for each case (as shown below). 

In [14]:
for c in sorted_k_partitions(['A','B','C','D'],2):
    print c

[('A',), ('B', 'C', 'D')]
[('B',), ('A', 'C', 'D')]
[('C',), ('A', 'B', 'D')]
[('D',), ('A', 'B', 'C')]
[('A', 'B'), ('C', 'D')]
[('A', 'C'), ('B', 'D')]
[('A', 'D'), ('B', 'C')]


In [15]:
for c in sorted_k_partitions(['A','B','C','D'],4):
    print c

[('A',), ('B',), ('C',), ('D',)]


In [16]:
for c in sorted_k_partitions(['A','B','C','D'],1):
    print c

[('A', 'B', 'C', 'D')]


### Optimizing Stops Among K People
I then write a function that wraps my single-person route optimization function. The function applies the original function to each unique way that the stops can be partitioned, and determines the the partition that minimizes the maximum time of the travelers. The input is identical to the original function except for the ```num_travelers``` parameter. If ```num_travelers``` is set to 1, the result will be identical between the two functions.

The output too is nearly identical to the previous function - a dictionary containing stop order, stop points, and trip time. However, now the function returns a list of these values, where the length of the list is equal to the number of travelers.

In [17]:
def optimize_stops_mult(home,stops,num_travelers,costing='pedestrian',api_key=matrix_key,home_label='Home',
                  stop_labels=None ):
    #get all possible options in which the stops can be broken up between the specified number of travelers
    options=sorted_k_partitions(stops,num_travelers)
    #create lists to store sublists of the time, name order, and point order for each option and traveler
    all_times=[]
    all_orders=[]
    all_points=[]
    all_lines=[]
    for option in options: #loop through each possible way to split the stops
        #create lists to store the time, name order, and point order for each traveler
        option_times=[] 
        option_orders=[]
        option_points=[]
        option_lines=[]
        #calculate the optimal stop order for each traveler with each set of stop options
        #using the previously defined function
        #append relevant information to lists
        for traveler in option:
            sub_labels=[stop_labels[stops.index(x)] for x in traveler] if stop_labels else None

            result=optimize_stops(home=home, stops=list(traveler), costing=costing, api_key=api_key,
                                  home_label=home_label, stop_labels=sub_labels)
            option_times.append(result['time'])
            option_orders.append(result['order'])
            option_points.append(result['points'])
            option_lines.append(result['line'])

        all_times.append(option_times)
        all_orders.append(option_orders)
        all_points.append(option_points)
        all_lines.append(option_lines)

    #get the index of the option that minimizes the max of any time that a traveler takes
    minloc=np.argmin([max(time) for time in all_times])
    
    #return a dictionary of the stop order, points, and time for the optimized options
    opt_order=[x[1:-1] for x in all_orders[minloc]]
    opt_times=all_times[minloc]
    opt_points=[x[1:-1] for x in all_points[minloc]]
    home_point=[x[0] for x in all_points[minloc]][0]
    polylines=all_lines[minloc]
    return {'order':opt_order,'time':opt_times, 'points':opt_points,'home_point':home_point, 'lines':polylines}


I use the same set of 6 stops in the Inner Richmond, but now specify that there will be two travelers. I extract the ordered set of stops for each of the two travelers, the time (in minutes) that each person's trip will take, and then plot the stops on a map.

As you can see, when I extract the stop order from the dictionary I get a list of two ordered sublists. This indicates that the first person should go to Pizzetta and Lamps Plus and the second person should go to Toy Boat, Noah's Bagels, Arguello, Market, and the de Young Museum (in that order). It may seem unfair that one person has to do 4 stops and the other only 2, but as you can see on the map, Pizzetta is much further away from home than the others. The person that makes the two stop trip will spend 53 minutes and the person that makes the four stop trip will spend 68 minutes. 68 minutes is the minimum amount of time that the person with the longer trip takes in any combination ways that these can be partitioned. 

In the map, the home location is shown in black, and each person's stops are shown in a different color. Stops are labeled with the stop name as well as the stop number (showing the order that the stops should be visited by each person).

In [18]:
opt_2_people=optimize_stops_mult(home, stops,num_travelers=2,stop_labels=stop_labels)

In [19]:
for i in opt_2_people['order']:
    print i

['Pizzetta', 'Lamps Plus']
['Toy Boat', "Noah's Bagels", 'Arguello Market', 'de Young Museum']


In [20]:
opt_2_people['time']


[53, 68]

In [34]:
points=[[opt_2_people['home_point']]]+opt_2_people['points']
labels=[['Home']]+[[' - '.join((str(i),x)) for i,x in enumerate(l,1)] for l in opt_2_people['order']]
m1=plot_stops(points, zoom_level=15, tiles='Stamen Watercolor',labels=labels,line_coords=opt_2_people['lines'])
m1

In [35]:
opt_2_people_bike=optimize_stops_mult(home, stops,num_travelers=2,stop_labels=stop_labels,costing='bicycle')
points=[[opt_2_people_bike['home_point']]]+opt_2_people_bike['points']
labels=[['Home']]+[[' - '.join((str(i),x)) for i,x in enumerate(l,1)] for l in opt_2_people_bike['order']]
m2=plot_stops(points, zoom_level=15, tiles='Stamen Watercolor',labels=labels,line_coords=opt_2_people_bike['lines'])
m2

Now let's say that Celeste and I have a friend who's willing to help us with our errands. I run the optimization function again, now specifying 3 travelers. As you can see, in the output lists and map below, one person will go to Pizzetta, one person will go to Toy Boat and Noah's Bagels, and one person will go to Lamp's Plus, the de Young, and Arguello Market. In this case, the person with the fewest amount of stops has the longest travel time (52 minutes), and the full set of errands will be made 15 minutes faster than it was with only 2 people.

In [23]:
opt_3_people=optimize_stops_mult(home, stops,3,stop_labels=stop_labels)

In [24]:
for i in opt_3_people['order']:
    print i

['Pizzetta']
['Toy Boat', "Noah's Bagels"]
['Lamps Plus', 'de Young Museum', 'Arguello Market']


In [25]:
opt_3_people['time']

[52, 44, 49]

In [36]:
points=[[opt_3_people['home_point']]]+opt_3_people['points']
labels=[['Home']]+[[' - '.join((str(i),x)) for i,x in enumerate(l,1)] for l in opt_3_people['order']]

m3=plot_stops(points, zoom_level=15, tiles='Stamen Watercolor',labels=labels,line_coords=opt_3_people['lines'])
m3

In [37]:
sf_points.save('sutro_transamerica.html')
sf_rand_points.save('sf_random_points.html')
m1.save('optimize_stops_2_people_walk.html')
m2.save('optimize_stops_2_people_bike.html')
m3.save('optimize_stops_3_people_walk.html')
